- import labeled data set
-- use all data, then take care about multiple similar looking images
- split into train / test sets
- train classifier
- test classifier

------------
ideas:
- train convnet on HOG features 
- train convnet on hog_image (?) -> does this make sense? 

In [1]:
import numpy as np 
import glob
import cv2

In [21]:

def read_images(image_dir_list):
    
    images = []
    for img_dir in image_dir_list:    
    
        # get all image files in the current directory
        img_file_list = glob.glob('./training_data/' + img_dir + '/?*.png')

        for img_file in img_file_list:
            img = cv2.imread(img_file)        
            images.append(img)
            #images.append(np.ravel(img))
            
    return images    
    
# define image data directories
vehicle_dirs = ['vehicles/GTI_Far', 'vehicles/GTI_Left', 'vehicles/GTI_MiddleClose', 
                'vehicles/GTI_Right'] #, 'vehicles/KITTI_extracted']
non_vehicle_dirs = ['non-vehicles/GTI']#, 'non-vehicles/Extras']

# read vehicle and non-vehicle images
vehicle_images = read_images(vehicle_dirs)
non_vehicle_images = read_images(non_vehicle_dirs)

        
print('number of vehicle images: {}'.format(len(vehicle_images)))   
print('number of non-vehicle images: {}'.format(len(non_vehicle_images)))   

number of vehicle images: 2826
number of non-vehicle images: 3900


In [41]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def get_hog_feature_map(img, plot_result=False):
    pix_per_cell = 8
    cell_per_block = 2
    num_orientations = 9     

    features, hog_image = hog(img, orientations=num_orientations, pixels_per_cell=(pix_per_cell, pix_per_cell), 
                              cells_per_block=(cell_per_block, cell_per_block), block_norm='L2-Hys',
                              visualise=True, feature_vector=False)

    if plot_result:
        fig = plt.figure()
        plt.subplot(121)
        plt.imshow(img, cmap='gray')
        plt.title('Example Car Image')
        plt.subplot(122)
        plt.imshow(hog_image, cmap='gray')
        plt.title('HOG Visualization')
        plt.show()
    
    return features, hog_image

def get_hog_feature_vec(img):
    pix_per_cell = 8
    cell_per_block = 2
    num_orientations = 9     

    feature_vec = hog(img, orientations=num_orientations, pixels_per_cell=(pix_per_cell, pix_per_cell), 
                      cells_per_block=(cell_per_block, cell_per_block), block_norm='L2-Hys',
                      visualise=False, feature_vector=True)

    return feature_vec

def plot_feature_vec(feature_vec):
    plt.bar(np.arange(0, len(feature_vec)), feature_vec)
    plt.show()
    
def normalize_vec(feature_vec):
    # Reshape the vector
    vector = feature_vec.reshape(-1, 1)
    # Create the scaler for normalization
    scaler = StandardScaler().fit(vector)
    # Apply the scaler 
    scaled_vec = scaler.transform(vector)
    
    return scaled_vec
        
def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def extract_features(images):
    
    features = []
    
    for img in images:
        img = convert_to_gray(img)
        feature_vec = get_hog_feature_vec(img)
        features.append(normalize_vec(feature_vec))
        
    return features



In [39]:
#img = vehicle_images[100]
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#feature_vec = get_hog_feature_vec(img)
#plot_feature_vec(feature_vec)

# extract features from images
vehicle_features = extract_features(vehicle_images)  
non_vehicle_features = extract_features(non_vehicle_images)  

In [85]:
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV

# create input data 'X' from features
X = np.vstack((vehicle_features, non_vehicle_features))
X = X.reshape((X.shape[0], X.shape[1]))
#print(X.shape)

# create output labels 'y': 1=vehicle, 0=non-vehicle
vehicle_labels = np.ones((len(vehicle_features)))
non_vehicle_labels = np.zeros((len(non_vehicle_features)))
y = np.hstack((vehicle_labels, non_vehicle_labels))
#print(y.shape)

# shuffle the data
X, y = shuffle(X, y)

# split data into training and test set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# train a SVM for classification by performing a grid search in parameter space
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 3, 5, 7, 9]}
svc = svm.SVC()
classifier = GridSearchCV(svc, parameters)
classifier.fit(X_train, y_train)  

print('finished training!')
print(classifier.best_params_) # {'C': 5, 'kernel': 'rbf'}


C:\Users\Tobias\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Tobias\AppData\Local\conda\conda\envs\carnd-term1\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


finished training!


AttributeError: 'SVC' object has no attribute 'best_params_'

In [88]:
def calculate_test_accuracy(features, labels):
    global classifier
    
    pred = classifier.predict(features)
    num_correct = np.count_nonzero(pred == y_test)
    acc = 1.0*num_correct/len(y_test)
            
    return acc
            

acc = calculate_test_accuracy(X_test, y_test)
print(acc)


0.9829123328380386


{'C': 5, 'kernel': 'rbf'}


[5 4 3 6] [2 1] [55 44 33 66] [22 11]
